In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# Read data from CSV file
file_name = 'phy_cps.csv'
data = pd.read_csv(file_name, index_col=0, header=0, parse_dates=True)

# Normalize the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Generate sequences and targets for LSTM
def create_sequences(data, n_timesteps):
    X, y = [], []
    for i in range(len(data) - n_timesteps):
        X.append(data[i:i+n_timesteps])
        y.append(data[i+n_timesteps])
    return np.array(X), np.array(y)

# Define parameters
n_timesteps = 10
n_features = data.shape[1]
latent_dim = 64
epochs = 50
batch_size = 32

# Prepare the dataset
X, y = create_sequences(data_scaled, n_timesteps)

# Split into training and test sets
split_idx = int(0.8 * len(X))
X_train, y_train = X[:split_idx], y[:split_idx]
X_test, y_test = X[split_idx:], y[split_idx:]

# Create LSTM model
model = Sequential()
model.add(LSTM(latent_dim, input_shape=(n_timesteps, n_features), return_sequences=True))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')

# Train model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

# Function to forecast iteratively
def forecast_iterative(model, input_seq, n_forecast_steps):
    input_seq = input_seq.copy()
    predictions = []
    for _ in range(n_forecast_steps):
        pred = model.predict(input_seq)
        predictions.append(pred[:, -1, :])
        input_seq = np.concatenate((input_seq[:, 1:, :], pred[:, -1:, :]), axis=1)
    return np.array(predictions).transpose(1, 0, 2)

# Number of steps to forecast
n_forecast_steps = 5

# Select the last sequence from the test set for forecasting
input_seq = X_test[-1:]

# Forecast iteratively
forecasted_values = forecast_iterative(model, input_seq, n_forecast_steps)

# Inverse transform the scaled data
forecasted_values_inv = scaler.inverse_transform(forecasted_values[0])
y_test_inv = scaler.inverse_transform(y_test[:n_forecast_steps])

# Flatten the true and predicted values for metric calculation
y_true = y_test_inv
y_pred = forecasted_values_inv

# Compute and print the metrics for each feature
for i in range(n_features):
    true = y_true[:, i]
    pred = y_pred[:, i]
    
    mse = mean_squared_error(true, pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(true, pred)
    r2 = r2_score(true, pred)
    
    print(f'Feature {data.columns[i]}:')
    print(f'MSE: {mse:.4f}')
    print(f'RMSE: {rmse:.4f}')
    print(f'MAE: {mae:.4f}')
    print(f'R2 Score: {r2:.4f}')
    print('')

# Plot the results (optional)
def plot_results(predictions, true_values, feature_name):
    plt.figure(figsize=(10, 6))
    plt.plot(true_values.flatten(), label='True')
    plt.plot(predictions.flatten(), label='Predicted')
    plt.xlabel('Time Step')
    plt.ylabel('Value')
    plt.title(f'{feature_name} Prediction vs True')
    plt.legend()
    plt.show()

# Plot each feature separately (optional)
for i in range(n_features):
    plot_results(y_pred[:, i], y_true[:, i], data.columns[i])


Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node 'mean_squared_error/SquaredDifference' defined at (most recent call last):
    File "C:\Users\user\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\user\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets\config\application.py", line 1053, in launch_instance
      app.start()
    File "C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\tornado\plat